#  **Can mental health professionals trust AI-based systems to prevent suicide? Effects of educational intervention and explanations on Trust**

**Developer:** Adonias Caetano de Oliveira

**Version:** Interface With XAI

## **Library installation and environment configuration**

In [ ]:
from google.colab import drive

PATH = '/content/drive'
drive.mount(PATH)

Mounted at /content/drive


In [ ]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=ce35847bb6e911df408b11ae8a453f5b2aa4f102aefeaf210e14b5ade9f263f8
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 118.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [ ]:
!pip install Unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## **Importing library**

In [ ]:
# Auxiliaries
import pandas as pd
import random
import time
import datetime
import numpy as np
import io
from scipy.special import expit
import re

# Deep learning and BERT
import torch
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from tqdm.notebook import tqdm

#NLP
from unidecode import unidecode
from string import punctuation

# Graph
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# XAI
import lime
from lime.lime_text import LimeTextExplainer

# Interface with Gradio
import gradio as gr

## **Reading the data set**

In [ ]:
url = '<link google drive of dataset CSV'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
dataset = pd.read_csv(read_url)

# display the first 5 rows
dataset.head()

,text,target
0,Aquela vontade de acabar com a minha vida voltou,1
1,to triste e com vontade de acabar com a minha ...,1
2,Corinthians ta querendo acabar com minha vida ...,0
3,Alguém poderia por favor me dar um tiro a acab...,1
4,TAYLOR TU VAI acabar com a minha vida MULHER,0


## **Text pre-processing**

In [ ]:
def clean(sentences):

  new_texts = []

  for text in sentences:
    text = text.lower()
    text = re.sub('@[^\s]+', '', text)
    text = unidecode(text)
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
    text = ''.join(c for c in text if c not in punctuation)
    new_texts.append(text)

  return new_texts

In [ ]:
dataset['text'] = clean(dataset['text'].values)

In [ ]:
def get_examples_sent():
  negativos = dataset.loc[dataset['target'] == 0].sample(n = 3)
  positivos = dataset.loc[dataset['target'] == 1].sample(n = 3)

  return list(negativos['text'].values) + list(positivos['text'].values)

In [ ]:
def get_examples_by_target(target, quant):
  return dataset.loc[dataset['target'] == target].sample(n = quant)

## **Classification with BERT**

In [ ]:
PRETRAINED_LM = 'neuralmind/bert-large-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

BertTokenizer(name_or_path='neuralmind/bert-large-portuguese-cased', vocab_size=29794, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [ ]:
def createDataloader(text):
  test_input_ids, test_att_masks = encode([text])
  BATCH_SIZE = 16
  test_y = torch.LongTensor([0])
  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)
  return test_dataloader

In [ ]:
labels_names = ['negativo', 'positivo']

In [ ]:
MODEL_PATH = PATH + '/My Drive/Colab Notebooks/Interface XAI - BERT - LIME/Models/model_95.bin'

def get_bert_model():
    N_labels = 2
    model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
    model.load_state_dict(torch.load(MODEL_PATH), strict=False)

    return model

model = get_bert_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
  # Tell pytorch to run this model on the GPU.
  model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

**Classifica um texto como POSITIVO ou NEGATIVO para ideação suicida**

In [ ]:
def predict(text):
  model.eval()
  test_dataloader = createDataloader(text)


  with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        logits = output.logits.cpu().detach().numpy()
        index_pred = np.argmax(logits,axis=-1)[0]
        probabilities = expit(logits)[0]


  df = pd.DataFrame(columns=['Label', 'Probabilidade'])
  df['Label'] = labels_names
  df['Probabilidade'] = probabilities

  return index_pred, labels_names[index_pred], df

**Retorna as probabilidades de um texto ser POSITIVO e NEGATIVO para ideação suicida**

In [ ]:
def predict_proba(sentences):
  model.eval()
  probabilities = []

  test_input_ids, test_att_masks = encode(sentences)
  BATCH_SIZE = 16
  test_y = torch.LongTensor([0] * len(sentences))
  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

  with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        probabilities.append(expit(output.logits.cpu().detach().numpy()))

  probabilities = np.concatenate(probabilities)

  return probabilities

## **Explanation of output with LIME**

In [ ]:
explainer = LimeTextExplainer(class_names = labels_names)

## **Interface XAI**

**Plota o resultado da classificação**

In [ ]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(y[i], i,str(f'{y[i]*100:0.2f}%'), ha = 'center', bbox = dict(facecolor = 'blue', alpha =.6))

def plot_classification(df):
    plt.clf()

    pos = np.arange(len(df['Label'].values))
    probabilidades = df['Probabilidade'].values

    fig = plt.barh(pos, probabilidades,color=df['Label'].map({'positivo': 'r', 'negativo': 'g'}),edgecolor='black')
    plt.yticks(pos, df['Label']) # Exibe cada token ou palavra no eixo y
    plt.legend(fig, [str(i) for i in ['Negativo', 'Positivo']]) # Exibe a legenda de cada classe (Positivo ou Negativo)
    plt.xlabel('Probabilities', fontsize=16)
    plt.ylabel('Class', fontsize=16)
    addlabels(pos, probabilidades) # calling the function to add value labels
    plt.title('Prediction probabilities',fontsize=16)
    plt.show()

    return plt

**Troca as cores do gráfico de LIME**

In [ ]:
def set_color_title(fig):
  # set title
  ax = fig.get_axes()[0]
  ax.set_title('Explicação Local para classe positivo')

  #set colors
  childrenLS=ax.get_children()
  barlist=[ x for x in childrenLS if isinstance(x, matplotlib.patches.Rectangle)]
  barlist.pop()

  for rectangle in barlist:

    position_x = rectangle.get_center()[0]

    if position_x < 0 and abs(position_x) > 0.001:
      rectangle.set_color('green')
    else:
      if position_x > 0.001:
        rectangle.set_color('red')

  return fig

**Retorna palavras categorizadas por classe**

In [ ]:
def get_categorized_words(lista):
  return [(tupla[0], "Positivo") if tupla[1] > 0 else (tupla[0], "Negativo") if tupla[1] < 0 else (tupla[0], "Neutro") for tupla in lista]

**Retorna todas as palavras como neutras**

In [ ]:
def get_neutral_words(text):
  tokens = text.split(" ")
  lista = []

  for token in tokens:
    lista.append((token, 0))

  return lista

**Retorna palavras na ordem correta da frase**

In [ ]:
def get_sorted_words(text, lista):
    tokens = text.split(" ")

    new_lista = []

    for token in tokens:
      is_added = False

      for tupla in lista:
        if token == tupla[0]:
          new_lista.append( (token, tupla[1]) )
          is_added = True

      if not tupla[0].strip() and not is_added:
        new_lista.append( (token, 0) )

    return new_lista

**Verifica se o texto é 100% negativo**

In [ ]:
# Palavras que podem indicm pensamentos suicidas
suicidal_word_list = ["suicida",  "suicídio", "sucidio", "matar", "bilhete", "carta",
             "despedida", "adeus", "acabar", "vida", "nunca", "acordar",
             "acorda", "não", "nao", "consigo", "continuar", "vale", "pena",
             "viver", "pronto", "para", "pular", "dormir", "sempre", "quero",
             "morrer", "morto", "melhor", "sem", "mim", "plano", "pacto",
             "cansado", "sozinho", "dormi", "sonho", "feliz", "só", "melhores",
             "ajuda", "valor", "entende", "bom", "escuridão", "ama", "amam", "confio", "passa", "passar", "vou", "outra"]

In [ ]:
def words_in_string(word_list, a_string):
    return set(word_list).intersection(a_string.split())

def is_negative(text):
  return not text or not text.strip() or not words_in_string(suicidal_word_list, text)

**Classsifica um texto**

In [ ]:
def classify_with_examples(text):

  if is_negative(text):
    info = f"<h4>O sistema verificou que o texto digitado não apresenta termos que podem indicar pensamento suicida!<br/>"
    info += f"Portanto, o sistema considerou o texto como 100% Negativo para ideação suicida.<br/>"
    info += f"Não foi aplicado inteligência artificial neste caso.<br/>"
    info += "Abaixo são exemplificadas frases que formam a base de conhecimento deste sistema.</h4>"
    result = {"Negativo": 1, "Positivo" : 0}
  else:
    # Classificação do texto
    index_pred, label, df = predict(text)
    d = df['Probabilidade'].to_dict()
    result = {"Negativo" if k == 0 else "Positivo" : v for k,v in d.items()}

    prob = df['Probabilidade'].max()

    info = f"<h4>O sistema classificou o texto como {labels_names[index_pred]} para ideação suicida com probabilidade = {prob*100:.2f}%. <br/>"
    info += "Abaixo são exemplificadas frases que formam a base de conhecimento deste sistema.</h4>"


  exemplos_negativos = get_examples_by_target(0, 5)
  exemplos_negativos.drop('target', axis=1, inplace=True)
  exemplos_negativos.rename(columns={"text": "Frases"}, inplace=True)

  exemplos_positivos = get_examples_by_target(1, 5)
  exemplos_positivos.drop('target', axis=1, inplace=True)
  exemplos_positivos.rename(columns={"text": "Frases"}, inplace=True)

  return result, exemplos_negativos, exemplos_positivos, info

### **INTERFACE COM EXPLICAÇÃO  E DIGITAÇÃO DE FRASES**

In [ ]:
INSTRUCAO_EXPLICACAO = "<h4>(1.) Cada palavra da frase possui uma pontuação de contribuição em relação à classe Positiva para ideação suicida.<br/>"
INSTRUCAO_EXPLICACAO += "(2.) A pontuação pode ser < 0 , ou seja, é uma palavra que no geral torna a frase como classe Negativa para ideação suicida.<br/>"
INSTRUCAO_EXPLICACAO += "(3.) Se a pontuação for > 0, ela contribui para classificar a frase como classe Positiva.<br/>"
INSTRUCAO_EXPLICACAO += "(4.) Palavras com pontuação zero não influenciam o sistema de classificação e, geralmente, são preposições e artigos'(Ex.: 'de',  'para', 'com', 'por', 'as', 'os', 'uns').<br/>"
INSTRUCAO_EXPLICACAO += "(5.) Somando essas pontuações ou scores, o método de explicação mostra como o modelo inteligente classificou o texto com base na soma final das contribuições positivas e negativas.<br/>"
INSTRUCAO_EXPLICACAO += "(6.) Abaixo é apresentada uma tabela de cada palavra e seu respectivo score e são exibidos dois gráficos de explicação da predição.<br/>"
INSTRUCAO_EXPLICACAO += "(7.) O primeiro gráfico destaca em cores verde (Negativo), branco (Neutro) e (Positivo) as palavras da frase.<br/>"
INSTRUCAO_EXPLICACAO += "(8.) O segundo gráfico a pontuação de cada palavra em relação à classe Positivo para ideação suicida através de barras horizontais.</h4>"
INSTRUCAO_EXPLICACAO

"<h4>(1.) Cada palavra da frase possui uma pontuação de contribuição em relação à classe Positiva para ideação suicida.<br/>(2.) A pontuação pode ser < 0 , ou seja, é uma palavra que no geral torna a frase como classe Negativa para ideação suicida.<br/>(3.) Se a pontuação for > 0, ela contribui para classificar a frase como classe Positiva.<br/>(4.) Palavras com pontuação zero não influenciam o sistema de classificação e, geralmente, são preposições e artigos'(Ex.: 'de',  'para', 'com', 'por', 'as', 'os', 'uns').<br/>(5.) Somando essas pontuações ou scores, o método de explicação mostra como o modelo inteligente classificou o texto com base na soma final das contribuições positivas e negativas.<br/>(6.) Abaixo é apresentada uma tabela de cada palavra e seu respectivo score e são exibidos dois gráficos de explicação da predição.<br/>(7.) O primeiro gráfico destaca em cores verde (Negativo), branco (Neutro) e (Positivo) as palavras da frase.<br/>(8.) O segundo gráfico a pontuação de cada

In [ ]:
def explain_text(text):
  if is_negative(text):
    info = f"<h4>Esse texto digitado foi considerado como 100% Negativo para ideação suicida!<br/>"
    info += f"Portanto, essa é a explicação do resultado.<br/>"
    lista = get_neutral_words(text)
    return info, pd.DataFrame(columns =['Palavra', 'Score']), get_categorized_words(lista), plt.figure()

  exp = explainer.explain_instance(text, classifier_fn = predict_proba, num_features = 10)
  lista = get_sorted_words( text, exp.as_list() )
  data_words = pd.DataFrame(lista, columns =['Palavra', 'Score'])
  fig = exp.as_pyplot_figure()
  return INSTRUCAO_EXPLICACAO, data_words, get_categorized_words(lista), set_color_title(fig)

**Interface de explicação com exemplos**

In [ ]:
with gr.Blocks() as demo_xai_digitacao:

  gr.Markdown(
    """
    # Interface de reconhecimento de ideação suicida com explicação dos resultados
    Comece a digitar abaixo para ver o resultado.
    """)

  input = gr.Textbox(label="Entrada:", placeholder="Digite ou selecione uma frase positiva ou negativa para ideação suicida...")


  gr.Examples(
        examples = get_examples_sent(),
        inputs = input
  )

  #Classificação
  classificar_btn = gr.Button("Classificar")
  result_classification = gr.Label(label="Classificação:")
  info_classification = gr.Markdown("Acione o botão classificar")

  frases_negativas = gr.Dataframe(
            label = "Exemplos de frases negativas para ideação suicida",
            headers=["Frases"],
            datatype=["str"],
            row_count=(5,"fixed"),
            col_count=(1, "fixed")
          )

  frases_positivas = gr.Dataframe(
            label = "Exemplos de frases positivas para ideação suicida",
            headers=["Frases"],
            datatype=["str"],
            row_count=(5,"fixed"),
            col_count=(1, "fixed")
          )

  classificar_btn.click(fn=classify_with_examples, inputs=input, outputs=[result_classification, frases_negativas, frases_positivas, info_classification] )

  # Explicação
  explicar_btn = gr.Button("Explicar")
  info_explaination = gr.Markdown("Acione o botão explicar")

  words_scores = gr.Dataframe(
          label = "Palavras e scores:",
          headers=["Palavra", "Score"],
          datatype=["str", "number"],
          row_count= (20,"fixed"),
          col_count=(2, "fixed")
  )

  plot_text_scores = gr.HighlightedText(
      label= "Texto original com as palavras categorizadas pela classe: Negativo < 0, Neutro = 0 e Positivo > 0.",
      combine_adjacent=False,
      show_legend=True,
      color_map={"Positivo": "red", "Negativo": "green", "Neutro": "white"}
    )

  plot_barh = gr.Plot(label = "Scores de cada palavra em relação à classe positiva")


  explicar_btn.click(fn=explain_text, inputs=input, outputs=[info_explaination, words_scores, plot_text_scores, plot_barh] )

demo_xai_digitacao.queue().launch(debug=True, share=True, inline=False)
#demo.launch(debug=True, share=True, inline=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://97be64fd32edb29ece.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://97be64fd32edb29ece.gradio.live
